### Libraries

In [ ]:
import pandas as pd
import numpy as np

### Functions

In [ ]:
def get_resampled_data(df_input: pd.DataFrame, resampling: str = 'W', date_col: str = 'inserted_datetime', device_col: str = 'deviceId', target: str = 'totalPowerGeneration') -> pd.DataFrame:
    """
    Resamples the input DataFrame by a specified frequency, aggregating the target variable for each device ID.

    Args:
        df_input (pd.DataFrame): Input DataFrame containing the data.
        resampling (str, optional): Resampling frequency. Defaults to 'W' (weekly), but can be changed to 'D' or 'M' or other options.
        date_col (str, optional): Column name of the date/time variable. Defaults to 'inserted_datetime'.
        device_col (str, optional): Column name of the device ID variable. Defaults to 'deviceId'.
        target (str, optional): Column name of the target variable to be aggregated. Defaults to 'totalPowerGeneration'.

    Returns:
        pd.DataFrame: Index: Resampled datetime with frequency=resampling, columns=  device IDs+ 'mean' (average value of devices), values= aggregated 'target' values for each device ID + average .
    """
    # Get unique device IDs
    ID_list = df_input[device_col].value_counts().keys()
  
    # Initialize empty DataFrame for resampled data
    df_resampled = pd.DataFrame()
  
    # Iterate over each device ID
    for id in ID_list:
        # Filter data for the current device ID
        df = df_input[df_input[device_col] == id]
      
        # Extract date and target columns
        df = df[[date_col, target]]
      
        # Set the date column as the index and convert it to datetime
        df.set_index(date_col, inplace=True)
        df.index = pd.to_datetime(df.index)
      
        # Sort the DataFrame by the index
        df.sort_index(inplace=True)
      
        # Resample the DataFrame by the specified frequency and calculate the mean
        df_w = df.resample(resampling).mean()
      
        # Add the resampled data to the result DataFrame, using the device ID as the column name
        df_resampled[f'{id}'] = df_w
  
    # Fill missing values with 0
    df_resampled.fillna(0, inplace=True)
  
    # Calculate the mean across all devices and add it as a new column
    df_resampled['mean'] = np.mean(df_resampled, axis=1)
  
    return df_resampled


In [ ]:


def find_abnormal_devices(dataframe: pd.DataFrame, threshold: float =0.7) -> pd.Series:
    """
    Identifies abnormal devices based on a threshold compared to the mean value in each row of the DataFrame.

    Args:
        dataframe (pd.DataFrame): Input DataFrame containing the data, which is the result of previous function (get_resampled_data).
        threshold (float, optional): Threshold value to identify abnormal devices. Defaults to 0.7.

    Returns:
        pd.Series: Series containing the abnormal devices, as a list for each row, with the row index indicating the timestamp.
    """
    # Create an empty Series to store the abnormal devices
    abnormal_df = pd.Series(index=dataframe.index)

    # Iterate over the rows of the DataFrame
    for index in dataframe.index:
        abnormal_devices = []

        # Check if the value for each device falls below the threshold relative to the mean value in the row
        for device in dataframe.columns[:-1]:
            if dataframe.loc[index, device] < (threshold * dataframe.loc[index, 'mean']):
                abnormal_devices.append(device)

        # Store the abnormal devices for the current row in the Series
        abnormal_df[index] = abnormal_devices

    return abnormal_df

In [ ]:
def Merge_Tracker_Inverter(df_Tracker: pd.DataFrame, df_inverters: pd.DataFrame, df_connection: pd.DataFrame, DeviceId: int, resampling: str = 'D', date_col='inserted_datetime', Target_name='dailyPowerGeneration', features=None):
    """
    Merges the Tracker and Inverter data based on the df_connection csv file for a specified DeviceId, resamples the data, and combines selected features plus a target.

    Args:
        df_Tracker (pd.DataFrame): DataFrame containing Tracker data.
        df_inverters (pd.DataFrame): DataFrame containing Inverter data.
        df_connection (pd.DataFrame): DataFrame containing the connection information between Trackers and Inverters.
        DeviceId (int): Device ID to merge the data for.
        resampling (str, optional): Resampling frequency. Defaults to 'D' (daily).
        date_col (str, optional): Column name of the date/time variable. Defaults to 'inserted_datetime'.
        Target_name (str, optional): Column name of the target variable to be resampled. Defaults to 'dailyPowerGeneration'.
        features (list, optional): List of column names of the features to be merged. If None, all columns except the first two will be considered. Defaults to None.

    Returns:
        pd.DataFrame: Merged and resampled DataFrame.
    """
    # Rename columns in the Tracker DataFrame
    df_Tracker.rename(columns={'TargetAngle_a1_rad': 'angle', 'Position_a1_rad': 'position', 'TempPcb_Kx10': 'Temp1', 'TempBat_Kx10': 'Temp2'}, inplace=True)
  
    # Set the default list of features
    if features is None:
        features = df_Tracker.columns.tolist()[2:]
  
    # Filter the Inverter DataFrame for the specified DeviceId
    df = df_inverters[df_inverters.deviceId == DeviceId]
    df = df.set_index(date_col)
    df.index = pd.to_datetime(df.index)
    df.sort_index(inplace=True)
  
    # Select the target column and resample it
    df_Idselected = df[[Target_name]]
    df_Idselected_res = df_Idselected.resample(resampling).mean().fillna(method="ffill").fillna(method="bfill").fillna(0)
  
    # Create the initial merged DataFrame
    df_Merge = df_Idselected_res.copy()
  
    # Modify the UniqueKey column in the Tracker DataFrame
    column_to_modify = 'UniqueKey'
    parts_to_remove = ['Archidona_1']
    for part in parts_to_remove:
        df_Tracker[column_to_modify] = df_Tracker[column_to_modify].str.replace(part, 'Archidona-1_')
  
    # Get the list of Tracker IDs for the specified DeviceId
    Tracker_list = df_connection['TrackerId'][df_connection['deviceid'] == DeviceId].unique().tolist()
  
    # Iterate over each feature
    for feature in features:
        # Create a copy of the Tracker DataFrame
        df_copy = df_Tracker.copy()
        df_copy['count'] = 1
        
        # Pivot the copied DataFrame
        df_pivot = df_copy.pivot_table(index=date_col, columns='UniqueKey', values=feature, aggfunc='sum')
        
        # Fill missing values and resample
        df_pivot = df_pivot.fillna(method="ffill").fillna(method='bfill').fillna(0)
        df_pivot = df_pivot.sort_index()
        df_pivot.index = pd.to_datetime(df_pivot.index)
        df_pivot_res = df_pivot.resample(resampling).mean().fillna(method="ffill")
        for i,col in enumerate(Tracker_list):
            df_pivot_res.rename(columns={Tracker_list[i]:f'{feature}_{i}'},inplace=True)
        df_Merge = pd.concat([df_Merge,df_pivot_res],axis=1)

    return df_Merge.fillna(0)